In [1]:
# Importações necessárias
from utils.Agente import AI_agente_extrator
from utils.pdf_to_img_to_text import extrair_texto_de_pdf_scaneado

from pathlib import Path


In [2]:
agent = AI_agente_extrator()

Modelo gemini-2.5-flash criado com sucesso.


In [3]:
agent.agent

Agent(model=GoogleModel(), name=None, end_strategy='early', model_settings=None, output_type=<class 'utils.BaseModel.ExtracaoOutput'>, instrument=None)

In [4]:
# Verificação da pasta de notas fiscais
pasta_notas = Path('nota_fiscal')
if not pasta_notas.exists():
    print(f"Pasta '{pasta_notas}' não encontrada.")

# Busca todos os arquivos PDF na pasta
arquivos_pdf = list(pasta_notas.glob('*.pdf'))
if not arquivos_pdf:
    print("Nenhum arquivo PDF encontrado na pasta 'nota_fiscal'.")

print(f"Encontrados {len(arquivos_pdf)} arquivos PDF para processar...")

Encontrados 1 arquivos PDF para processar...


In [5]:
arquivo_txt = extrair_texto_de_pdf_scaneado(arquivos_pdf[0])
arquivo_txt

PDF tem 1 página(s). Processando...
Processando página 1...
Processamento concluído.


"--- PÁGINA 1 ---\nRPS Nº 4552062 Série RPS, emitido em 30/09/2025| Numero da Nota\nPREFEITURA DO 04365344\n\nLemes Data e Hora de Emissão\nR E C I F E e 02/10/2025 15:12:19\nSECRETARIA DE FINANÇAS Código de Verificação\n\nNota Fiscal de Serviços Eletrônica IC44-CPYF\nPRESTADOR DE SERVIÇOS\nCPF/CNPJ: 04.986.320/0031-39 Inscrição Municipal: 511.256-7\nNome/Razão Social: SER EDUCACIONAL S.A.\nEndereço: AVENIDA GUARARAPES 283, LOJA 3 - SANTO ANTONIO - CEP: 50010-000\nMunicípio: Recife UF: PE E-mail: contabilidade()sereducacional.com\n\nTOMADOR DE SERVIÇOS\nNome/Razão Social: ALEX SILVA DE ASSUNÇÃO\n\ncrrCNPJDD O Inscrição Municipal: --—-\nEndereçETTTTTI\n\nMunicípio: Curitiba ur: PR E-mail: alexsaweb 6) gmail.com\n\nDISCRIMINAÇÃO DOS SERVIÇOS\n\nMENSALIDADE EAD\n\nVALOR TOTAL DO SERVIÇO = R$ 138,33\n\nCódigo da Atividade Prestada a . .\n8532500 - EDUCAÇÃO SUPERIOR - GRADUAÇÃO E POS-GRADUAÇÃO\n\n08.01 - Ensino regular pré escolar, fundamental, médio e superior.\n\nDeduções (R$) Desconto In

In [6]:
resultado_txt = await agent.extrair_dados_txt(arquivo_txt)

In [7]:
resultado_txt

'{"descricao_servico":"MENSALIDADE EAD","valor_servico":138.33,"numero_nf":4365344,"data_emissao":"2025-10-02","valor_total":138.33,"cnpj":"04986320003139"}'

In [8]:
pdf_nota = 'nota_fiscal/NFSe3.pdf'
arquivobyte = Path(pdf_nota).read_bytes()

In [9]:
resultado_pdf = await agent.extrair_dados_img(arquivobyte)

In [10]:
resultado_pdf

'{"descricao_servico":"MENSALIDADE EAD","valor_servico":138.33,"numero_nf":4365344,"data_emissao":"2025-10-02","valor_total":138.33,"cnpj":"04986320003139"}'